# Rf 4 0 4_Categories
Data and categories: working with RooCategory objects to describe discrete variables



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:52 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooPolynomial.h"
#include "RooCategory.h"
#include "Roo1DTable.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Construct  a category with labels
 ----------------------------------------------------------------

Define a category with labels only

In [3]:
RooCategory tagCat("tagCat", "Tagging category");
tagCat.defineType("Lepton");
tagCat.defineType("Kaon");
tagCat.defineType("NetTagger-1");
tagCat.defineType("NetTagger-2");
tagCat.Print();


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

RooCategory::tagCat = Lepton(idx = 0)



Construct  a category with labels and indeces
 ----------------------------------------------------------------------------------------

Define a category with explicitly numbered states

In [4]:
RooCategory b0flav("b0flav", "B0 flavour eigenstate");
b0flav.defineType("B0", -1);
b0flav.defineType("B0bar", 1);
b0flav.Print();

RooCategory::b0flav = B0(idx = -1)



Generate dummy data for tabulation demo
 ----------------------------------------------------------------------------

Generate a dummy dataset

In [5]:
RooRealVar x("x", "x", 0, 10);
RooDataSet *data = RooPolynomial("p", "p", x).generate(RooArgSet(x, b0flav, tagCat), 10000);

Print tables of category contents of datasets
 ------------------------------------------------------------------------------------------

Tables are equivalent of plots for categories

In [6]:
Roo1DTable *btable = data->table(b0flav);
btable->Print();
btable->Print("v");

Roo1DTable::b0flav = (B0=5058,B0bar=4942)

  Table b0flav : pData
  +-------+------+
  |    B0 | 5058 |
  | B0bar | 4942 |
  +-------+------+



Create table for subset of events matching cut expression

In [7]:
Roo1DTable *ttable = data->table(tagCat, "x>8.23");
ttable->Print();
ttable->Print("v");

Roo1DTable::tagCat = (Lepton=454,Kaon=450,NetTagger-1=432,NetTagger-2=429)

  Table tagCat : pData(x>8.23)
  +-------------+-----+
  |      Lepton | 454 |
  |        Kaon | 450 |
  | NetTagger-1 | 432 |
  | NetTagger-2 | 429 |
  +-------------+-----+



Create table for all (tagcat x b0flav) state combinations

In [8]:
Roo1DTable *bttable = data->table(RooArgSet(tagCat, b0flav));
bttable->Print("v");


  Table (tagCat x b0flav) : pData
  +---------------------+------+
  |         {Lepton;B0} | 1302 |
  |           {Kaon;B0} | 1232 |
  |    {NetTagger-1;B0} | 1242 |
  |    {NetTagger-2;B0} | 1282 |
  |      {Lepton;B0bar} | 1192 |
  |        {Kaon;B0bar} | 1314 |
  | {NetTagger-1;B0bar} | 1208 |
  | {NetTagger-2;B0bar} | 1228 |
  +---------------------+------+



Retrieve number of events from table
 Number can be non-integer if source dataset has weighed events

In [9]:
Double_t nb0 = btable->get("B0");
cout << "Number of events with B0 flavor is " << nb0 << endl;

Number of events with B0 flavor is 5058


Retrieve fraction of events with "lepton" tag

In [10]:
Double_t fracLep = ttable->getFrac("Lepton");
cout << "Fraction of events tagged with Lepton tag is " << fracLep << endl;

Fraction of events tagged with Lepton tag is 0.257224


Defining ranges for plotting, fitting on categories
 ------------------------------------------------------------------------------------------------------

Define named range as comma separated list of labels

In [11]:
tagCat.setRange("good", "Lepton,Kaon");

Or add state names one by one

In [12]:
tagCat.addToRange("soso", "NetTagger-1");
tagCat.addToRange("soso", "NetTagger-2");

Use category range in dataset reduction specification

In [13]:
RooDataSet *goodData = (RooDataSet *)data->reduce(CutRange("good"));
goodData->table(tagCat)->Print("v");


  Table tagCat : pData
  +-------------+------+
  |      Lepton | 2494 |
  |        Kaon | 2546 |
  | NetTagger-1 |    0 |
  | NetTagger-2 |    0 |
  +-------------+------+

